In [1]:
import numpy as np
import pandas as pd

from IPython.display import Image

from sklearn import datasets
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn import svm

from matplotlib import pyplot as plt
%matplotlib inline

In [4]:
df = pd.read_csv('sea_2018.csv')
df.head()

,Unnamed: 0,game_date,player_name,batter,pitcher,zone,home_team,away_team,balls,strikes,...,count_2-0,count_2-1,count_2-2,count_3-0,count_3-1,count_3-2,fastball,run_diff,breaking,offspeed
0,0,9/30/18,Shawn Armstrong,593643,542888,13,SEA,TEX,2,2,...,0,0,1,0,0,0,1,-2,0,0
1,1,9/30/18,Shawn Armstrong,593643,542888,14,SEA,TEX,2,2,...,0,0,1,0,0,0,1,-2,0,0
2,2,9/30/18,Shawn Armstrong,593643,542888,8,SEA,TEX,2,2,...,0,0,1,0,0,0,1,-2,0,0
3,3,9/30/18,Shawn Armstrong,593643,542888,9,SEA,TEX,2,2,...,0,0,1,0,0,0,0,-2,1,0
4,4,9/30/18,Shawn Armstrong,593643,542888,3,SEA,TEX,2,2,...,0,0,1,0,0,0,1,-2,0,0


In [42]:
df['pitch']=0

In [6]:
df.head()

,Unnamed: 0,game_date,player_name,batter,pitcher,zone,home_team,away_team,balls,strikes,...,count_2-1,count_2-2,count_3-0,count_3-1,count_3-2,fastball,run_diff,breaking,offspeed,pitch
0,0,9/30/18,Shawn Armstrong,593643,542888,13,SEA,TEX,2,2,...,0,1,0,0,0,1,-2,0,0,0
1,1,9/30/18,Shawn Armstrong,593643,542888,14,SEA,TEX,2,2,...,0,1,0,0,0,1,-2,0,0,0
2,2,9/30/18,Shawn Armstrong,593643,542888,8,SEA,TEX,2,2,...,0,1,0,0,0,1,-2,0,0,0
3,3,9/30/18,Shawn Armstrong,593643,542888,9,SEA,TEX,2,2,...,0,1,0,0,0,0,-2,1,0,0
4,4,9/30/18,Shawn Armstrong,593643,542888,3,SEA,TEX,2,2,...,0,1,0,0,0,1,-2,0,0,0


In [11]:
df['pitch'][df.fastball == 1] = 2

/Users/alexanderbell/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [21]:
df.sample()

,game_date,player_name,batter,pitcher,zone,home_team,away_team,balls,strikes,on_3b,on_2b,on_1b,inning,sv_id,pitch_name,bat_score,fld_score,pitch_type_CH,pitch_type_CU,pitch_type_FC,pitch_type_FF,pitch_type_FS,pitch_type_FT,pitch_type_KC,pitch_type_PO,pitch_type_SI,pitch_type_SL,p_throws_L,p_throws_R,stand_L,stand_R,outs_when_up_0,outs_when_up_1,outs_when_up_2,if_shift,if_standard,if_strategic,of_fourth,of_standard,of_strategic,count_0-0,count_0-1,count_0-2,count_1-0,count_1-1,count_1-2,count_2-0,count_2-1,count_2-2,count_3-0,count_3-1,count_3-2,fastball,run_diff,breaking,offspeed,pitch
22243,4/5/18,Dan Altavilla,621439,656186,12,MIN,SEA,0,1,0,0,0,6,180405_221834,Slider,2,2,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,1,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1


In [13]:
df['pitch'][df.breaking == 1] = 1

/Users/alexanderbell/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [15]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [17]:
df.drop('Unnamed: 0',axis=1,inplace=True)

In [22]:
X = df[['on_3b','on_2b','on_1b','inning','p_throws_L','stand_L','if_shift','if_strategic','of_fourth',
        'of_strategic','count_0-1','count_0-2','count_1-0','count_1-1','count_1-2','count_2-0','count_2-1',
        'count_2-2','count_3-0','count_3-1','count_3-2','run_diff','outs_when_up_1','outs_when_up_2']]

In [23]:
y = df.pitch

In [24]:
feat = np.array(X)

In [25]:
obs = np.ravel(y)

In [26]:
feat

array([[ 0,  0,  0, ..., -2,  0,  1],
       [ 0,  0,  0, ..., -2,  0,  1],
       [ 0,  0,  0, ..., -2,  0,  1],
       ..., 
       [ 0,  0,  0, ...,  0,  1,  0],
       [ 0,  0,  0, ...,  0,  0,  0],
       [ 0,  0,  0, ...,  0,  0,  0]])

In [27]:
obs

array([2, 2, 2, ..., 2, 2, 2])

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5,random_state=42)

In [29]:
pca = PCA(n_components=2)
pca.fit(X_train)

PCA(copy=True, iterated_power='auto', n_components=2, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

In [30]:
pcafeatures_train = pca.transform(X_train)

In [31]:
from itertools import cycle

def plot_PCA_2D(data, target, target_names):
    colors = cycle(['r','g','b'])
    target_ids = range(len(target_names))
    plt.figure()
    for i, c, label in zip(target_ids, colors, target_names):
        plt.scatter(data[target == i, 0], data[target == i, 1],
                   c=c, label=label)
    plt.legend()

In [36]:
df.shape

(23065, 57)

In [38]:
# fit linear model
model_svm = svm.SVC(kernel='linear')
model_svm.fit(X_train, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [39]:
# predict out of sample
y_pred = model_svm.predict(X_test)

In [40]:
# check accuracy
accuracy_score(y_test,y_pred)
# pretty good accuracy!

0.63235931674325851

In [41]:
# confusion matrix
confusion_matrix(y_test,y_pred)

array([[   0,    0, 1612],
       [   0,    0, 2628],
       [   0,    0, 7293]])